Perform a random walk sampling of this network. Sample 2,000 nodes (1% of the network) and all their connections (note: the sample will end up having more than 2,000 nodes).

In [1]:
import random
import networkx as nx

G = nx.read_edgelist("data.txt", nodetype = int)

In [2]:
# We need to keep track of the nodes and the edges we have sampled, so we store them in sets.
# Then we need a seed node where to start the exploration. Here I decided to just pick a random node.
sampled_nodes = set()
sampled_edges = set()
curnode = random.choice(list(G.nodes))

In [3]:
# We continue until we sampled 2000 nodes.
while len(sampled_nodes) <= 2000:
   # First we get the neighbors of the node we're currently exploring
   neighbors = list(G.neighbors(curnode))
   if not curnode in sampled_nodes: # This is true if we never sampled this node before. This means we never added its connections to sampled_edges
      sampled_nodes.add(curnode) # This will allow us to remember we sampled this node
      # We update the set of sampled edges. We need to have a canonical representation of the edge because the network is undirected,
      # so if we already saw the edge because we sampled the neighbor, we might have stored the edge as (neighbor, curnode) rather than
      # (curnode, neighbor). With this min-max trick, this is not an issue.
      sampled_edges |= set([(min(curnode, neighbor), max(curnode, neighbor)) for neighbor in neighbors])
   curnode = random.choice(neighbors) # We move on to sampling a random neighbor of the current node, because we're doing a r

In [4]:
G_smpl = nx.Graph(list(sampled_edges))
print(len(G_smpl.nodes))

c:\Users\jackh\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:177: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


62732


Compare the CCDF of the degree distribution of your sample of the network from Exercise 25.1 with the one of the original network by fitting a log-log regression and comparing the exponents. You can take multiple samples from different seeds to ensure the robustness of your result.

In [9]:
import random
import numpy as np
import pandas as pd
import networkx as nx
from collections import Counter
from scipy.stats import linregress

# Function implementing the random walk logic from the previous question
def rw(G, n):
   sampled_nodes = set()
   sampled_edges = set()
   curnode = random.choice(list(G.nodes))
   while len(sampled_nodes) <= n:
      neighbors = list(G.neighbors(curnode))
      if not curnode in sampled_nodes:
         sampled_nodes.add(curnode)
         sampled_edges |= set([(min(curnode, neighbor), max(curnode, neighbor)) for neighbor in neighbors])
      curnode = random.choice(neighbors)
   return nx.Graph(list(sampled_edges))

# Function generating a CCDF, from previous exercises
def ccdf(dd):
   dd = pd.DataFrame(list(dd.items()), columns = ("k", "count")).sort_values(by = "k")
   ccdf = dd.sort_values(by = "k", ascending = False)
   ccdf["cumsum"] = ccdf["count"].cumsum()
   ccdf["ccdf"] = ccdf["cumsum"] / ccdf["count"].sum()
   ccdf = ccdf[["k", "ccdf"]].sort_values(by = "k")
   return ccdf

# Function performing a simple regression in log-log space
def dd_exponent(degdistr):
   logcdf = np.log10(degdistr[["k", "ccdf"]])
   slope, log10intercept, r_value, p_value, std_err = linregress(logcdf["k"], logcdf["ccdf"])
   return slope

In [10]:
G = nx.read_edgelist("data.txt", nodetype = int)
G_ccdf = ccdf(Counter(dict(G.degree).values()))
print("Original Exponent: %1.4f" % dd_exponent(G_ccdf))

Original Exponent: -1.6013


In [11]:
# Let's take 100 samples and store their degree exponent in a list
smpl_exponents = []
for _ in range(100):
   G_smpl = rw(G, 2000)
   G_smpl_ccdf = ccdf(Counter(dict(G_smpl.degree).values()))
   smpl_exponents.append(dd_exponent(G_smpl_ccdf))

In [12]:
smpl_exponents_mean = np.mean(smpl_exponents)
smpl_exponents_std = np.std(smpl_exponents)
print("Sample Exponent: %1.4f (+/- %1.4f)" % (smpl_exponents_mean, smpl_exponents_std)) # The exponent of the sample is diffe

Sample Exponent: -1.1259 (+/- 0.0104)


Modify the degree distribution of your sample of the network from Exercise 25.1 using the Re-Weighted Random Walk technique. Is the estimation of the exponent of the CCDF more precise?

In [15]:
import random
import numpy as np
import pandas as pd
import networkx as nx
from collections import Counter
from scipy.stats import linregress

# Function implementing the random walk logic from the previous question
def rw(G, n):
   sampled_nodes = set()
   sampled_edges = set()
   curnode = random.choice(list(G.nodes))
   while len(sampled_nodes) <= n:
      neighbors = list(G.neighbors(curnode))
      if not curnode in sampled_nodes:
         sampled_nodes.add(curnode)
         sampled_edges |= set([(min(curnode, neighbor), max(curnode, neighbor)) for neighbor in neighbors])
      curnode = random.choice(neighbors)
   return nx.Graph(list(sampled_edges))

# Function implementing the RWRW correction
def rwrw(dd_smpl, nnodes):
   dd_smpl = pd.DataFrame(dd_smpl, columns = ("k",))
   dd_smpl["k_inv"] = 1.0 / dd_smpl["k"] # We need to calculate all x^{-1} values
   denominator = dd_smpl["k_inv"].sum() # The denominator of RWRW is simply their sum
   dd_smpl = dd_smpl.groupby(by = "k").sum().reset_index() # For every value in the degree distribution, we need the sum of its inverse (numerator)
   dd_smpl["p"] = dd_smpl["k_inv"] / denominator # Making the fraction
   # So far we have the probability of observing a value. We want instead the number of nodes with that value.
   # So we multiply the probability by the number of nodes in the sample.
   dd_smpl["count"] = dd_smpl["p"] * nnodes
   # Now we generate and return the CCDF as usual
   ccdf = dd_smpl.sort_values(by = "k", ascending = False)
   ccdf["cumsum"] = ccdf["count"].cumsum()
   ccdf["ccdf"] = ccdf["cumsum"] / ccdf["count"].sum()
   ccdf = ccdf[["k", "ccdf"]].sort_values(by = "k")
   return ccdf

# Function generating a CCDF, from previous exercises
def ccdf(dd):
   dd = pd.DataFrame(list(dd.items()), columns = ("k", "count")).sort_values(by = "k")
   ccdf = dd.sort_values(by = "k", ascending = False)
   ccdf["cumsum"] = ccdf["count"].cumsum()
   ccdf["ccdf"] = ccdf["cumsum"] / ccdf["count"].sum()
   ccdf = ccdf[["k", "ccdf"]].sort_values(by = "k")
   return ccdf

# Function performing a simple regression in log-log space
def dd_exponent(degdistr):
   logcdf = np.log10(degdistr[["k", "ccdf"]])
   slope, log10intercept, r_value, p_value, std_err = linregress(logcdf["k"], logcdf["ccdf"])
   return slope

In [17]:
G = nx.read_edgelist("data.txt", nodetype = int)
G_ccdf = ccdf(Counter(dict(G.degree).values()))
print("Original Exponent: %1.4f" % dd_exponent(G_ccdf))

Original Exponent: -1.6013


In [18]:
# Let's take 50 samples and store their degree exponent in a list
smpl_exponents = []
for _ in range(100):
   G_smpl = rw(G, 2000)
   G_smpl_ccdf = rwrw(list(dict(G_smpl.degree()).values()), len(G_smpl.nodes))
   smpl_exponents.append(dd_exponent(G_smpl_ccdf))

In [19]:
smpl_exponents_mean = np.mean(smpl_exponents)
smpl_exponents_std = np.std(smpl_exponents)
print("Sample Exponent: %1.4f (+/- %1.4f)" % (smpl_exponents_mean, smpl_exponents_std)) # The correction moves the RW sample exponent from ~1.1 to ~2.1

Sample Exponent: -2.0982 (+/- 0.0129)


Modify your random walk sampler for the network from Exercise 25.1 so that it applies the Metropolis-Hastings correction. Is the estimation of the exponent of the CCDF more precise? Is MHRW more or less precise than RWRW?

In [20]:
G = nx.read_edgelist("data.txt", nodetype = int)
G_ccdf = ccdf(Counter(dict(G.degree).values()))
print("Original Exponent: %1.4f" % dd_exponent(G_ccdf))

Original Exponent: -1.6013


In [23]:
# Function implementing the random walk logic from the previous question, plus the Metropolis-Hastings correction
def mhrw(G, n):
   sampled_nodes = set()
   sampled_edges = set()
   curnode = random.choice(list(G.nodes))
   while len(sampled_nodes) <= n:
      neighbors = list(G.neighbors(curnode))
      if not curnode in sampled_nodes:
         sampled_nodes.add(curnode)
         sampled_edges |= set([(min(curnode, neighbor), max(curnode, neighbor)) for neighbor in neighbors])
      degree_x = len(neighbors) # We need to know the degree of the neighbor we're currently on
      candidate = random.choice(neighbors) # We draw a potential node to visit at the next step
      # We keep rejecting candidates if we extract a random number higher than the ratio between current node degree and candidate degree
      while random.random() >= (degree_x / G.degree(candidate)): 
         candidate = random.choice(neighbors)
      curnode = candidate
   return nx.Graph(list(sampled_edges))

In [24]:
# Let's take 100 samples and store their degree exponent in a list
smpl_exponents = []
for _ in range(100):
   G_smpl = mhrw(G, 2000)
   G_smpl_ccdf = ccdf(Counter(dict(G_smpl.degree).values()))
   smpl_exponents.append(dd_exponent(G_smpl_ccdf))

In [25]:
smpl_exponents_mean = np.mean(smpl_exponents)
smpl_exponents_std = np.std(smpl_exponents)
print("Sample Exponent: %1.4f (+/- %1.4f)" % (smpl_exponents_mean, smpl_exponents_std)) # The exponent of the sample is ~1.4, much closer to the original 1.6 than either RW and RWRW

Sample Exponent: -1.3857 (+/- 0.0633)
